In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns

## Read data

In [2]:
df = pd.read_csv("./output/all-data.csv")

### Get concise summary of the dataframe 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [4]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


## Data Cleaning

### First: Column labels

In [5]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

##### We notice two problems with column labels:
1. They are long.
2. There is a space in each label.

#### Rename

In [6]:
df.rename(columns={"Quantity Ordered":"Quantity",
                   "Price Each":"unit price","Order Date":"Date",
                   "Purchase Address":"Address"},inplace=True)

#### Capitalize

In [7]:
df.rename(str.title, axis='columns',inplace=True)

#### Remove spaces 

In [8]:
df.columns=df.columns.str.replace(" ","")

##### Let's look at them now

In [9]:
df.columns

Index(['OrderId', 'Product', 'Quantity', 'UnitPrice', 'Date', 'Address'], dtype='object')

### Second: Cast data types

#### Convert ```"OrderId","Quantity","Price"``` columns to numeric values

In [10]:
df[["OrderId","Quantity","UnitPrice"]]=df[["OrderId","Quantity","UnitPrice"]].apply(pd.to_numeric,errors="coerce",downcast ='integer')

#### Convert ```"Date"``` column to datetime object

In [11]:
df["Date"] = pd.to_datetime(df["Date"],format = "%d/%m/%y %H:%M",errors="coerce")

#### Drop NaN values

In [12]:
df.dropna(inplace=True)

#### Convert ```"OrderId","Quantity"``` columns to integer
The data type of these columns is **float** because there were **NaN values** at them, but after dropping them in the last step, we can now convert the columns into **integer**.

In [13]:
df[["OrderId","Quantity"]].dtypes

OrderId     float64
Quantity    float64
dtype: object

In [14]:
convert_dict = {'OrderId': int,
                'Quantity': int
               }
  
df = df.astype(convert_dict)

### Third: Set the ```"Date"``` column as index

In [15]:
df.set_index("Date",inplace=True)
df.sort_index(inplace=True) #sort rows by their date

In [16]:
df.head()

,OrderId,Product,Quantity,UnitPrice,Address
Date,,,,,
2019-01-01 03:07:00,147268,Wired Headphones,1,11.99,"9 Lake St, New York City, NY 10001"
2019-01-01 03:40:00,148041,USB-C Charging Cable,1,11.95,"760 Church St, San Francisco, CA 94016"
2019-01-01 04:56:00,149343,Apple Airpods Headphones,1,150.00,"735 5th St, New York City, NY 10001"
2019-01-01 05:53:00,149964,AAA Batteries (4-pack),1,2.99,"75 Jackson St, Dallas, TX 75001"
2019-01-01 06:03:00,149350,USB-C Charging Cable,2,11.95,"943 2nd St, Atlanta, GA 30301"


### Fourth: Add new columns

#### Add ```"Month"``` column

In [17]:
df["Month"]=df.index.month_name()

#### Add ```"DayOrder"``` column

In [18]:
df["DayOrder"]=df.index.day

#### Add ```"DayName"``` column

In [19]:
df["DayName"]=df.index.day_name()

#### Add ```"TotalCost"``` column 

In [20]:
df["TotalCost"] = df["Quantity"] * df["UnitPrice"]

#### Add ```"City"``` column

In [21]:
df["Address"].head(3)

Date
2019-01-01 03:07:00        9 Lake St, New York City, NY 10001
2019-01-01 03:40:00    760 Church St, San Francisco, CA 94016
2019-01-01 04:56:00       735 5th St, New York City, NY 10001
Name: Address, dtype: object

##### 1. Split the address column

In [22]:
splited_address = df["Address"].str.split(", ",expand=True)
splited_address.head(3)

,0,1,2
Date,,,
2019-01-01 03:07:00,9 Lake St,New York City,NY 10001
2019-01-01 03:40:00,760 Church St,San Francisco,CA 94016
2019-01-01 04:56:00,735 5th St,New York City,NY 10001


##### 2. Split the  (state and postal code) column

In [23]:
splited_state_postalcode = splited_address[2].str.split(expand=True)
splited_state_postalcode.head(3)

,0,1
Date,,
2019-01-01 03:07:00,NY,10001
2019-01-01 03:40:00,CA,94016
2019-01-01 04:56:00,NY,10001


##### 3. Merge states and cities in one column

In [24]:
cities = splited_address[1]
states = splited_state_postalcode[0]

merged_city_state = cities.str.cat(states,sep=", ")
merged_city_state.head(3)

Date
2019-01-01 03:07:00    New York City, NY
2019-01-01 03:40:00    San Francisco, CA
2019-01-01 04:56:00    New York City, NY
Name: 1, dtype: object

##### 4. Add the ```"City"``` column

In [25]:
df["City"] = merged_city_state

In [26]:
df.head()

,OrderId,Product,Quantity,UnitPrice,Address,Month,DayOrder,DayName,TotalCost,City
Date,,,,,,,,,,
2019-01-01 03:07:00,147268,Wired Headphones,1,11.99,"9 Lake St, New York City, NY 10001",January,1,Tuesday,11.99,"New York City, NY"
2019-01-01 03:40:00,148041,USB-C Charging Cable,1,11.95,"760 Church St, San Francisco, CA 94016",January,1,Tuesday,11.95,"San Francisco, CA"
2019-01-01 04:56:00,149343,Apple Airpods Headphones,1,150.00,"735 5th St, New York City, NY 10001",January,1,Tuesday,150.00,"New York City, NY"
2019-01-01 05:53:00,149964,AAA Batteries (4-pack),1,2.99,"75 Jackson St, Dallas, TX 75001",January,1,Tuesday,2.99,"Dallas, TX"
2019-01-01 06:03:00,149350,USB-C Charging Cable,2,11.95,"943 2nd St, Atlanta, GA 30301",January,1,Tuesday,23.90,"Atlanta, GA"
